《[利用Python进行数据分析](https://book.douban.com/subject/25779298/)》读书笔记。
 
 [第10章](/2017/07/20/python_data_analysis10.html)  第4节：时区处理

所有用到的数据可以从[作者的 github](https://github.com/wesm/pydata-book)下载。


In [1]:
%pylab inline
import pandas as pd
from datetime import datetime
from pandas import Series, DataFrame

Populating the interactive namespace from numpy and matplotlib


时间序列最让人不爽的就是对时区的处理。很多人已经用协调世界时（UTC，格林尼治时间接替者，目前是国际标准）来处理时间序列。

时区就是以UTC偏移量的形式表示的。

Python中，时区信息来自第三方库pytz，它可以使Python可以使用Olson数据库。

pandas包装了pytz功能。因此不用记忆API，只要记得时区名称即可。时区名可以在文档中找到。


In [5]:
# 通过交互的方式查看时区
import pytz
pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

In [6]:
tz = pytz.timezone('Asia/Shanghai')
tz

<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>

## 本地化和转换

默认情况下，pandas中的序列是单纯的（naive）时区，其索引的tz字段为None.

In [7]:
rng = pd.date_range('3/9/2012 9:30', periods=6, freq='D')
ts = Series(np.random.randn(len(rng)), index=rng)

In [9]:
print(ts.index.tz)

None


In [11]:
# 在生成日期范围的时候可以加上一个时区集
print(pd.date_range('3/9/2012',periods = 10,freq = 'D',tz = 'UTC'))

DatetimeIndex(['2012-03-09', '2012-03-10', '2012-03-11', '2012-03-12',
               '2012-03-13', '2012-03-14', '2012-03-15', '2012-03-16',
               '2012-03-17', '2012-03-18'],
              dtype='datetime64[ns, UTC]', freq='D')


In [14]:
# 转换时区是通过tz_localize方法处理的
ts_utc = ts.tz_localize('UTC')
ts_utc

2012-03-09 09:30:00+00:00    1.570592
2012-03-10 09:30:00+00:00    1.514111
2012-03-11 09:30:00+00:00   -0.818680
2012-03-12 09:30:00+00:00   -0.097930
2012-03-13 09:30:00+00:00   -0.072169
2012-03-14 09:30:00+00:00    0.141429
Freq: D, dtype: float64

In [15]:
ts_utc.index

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [16]:
# 一旦被转换为某个特定时期，就可以用tz_convert将其转换到其他时区了
ts_utc.tz_convert('US/Eastern')

2012-03-09 04:30:00-05:00    1.570592
2012-03-10 04:30:00-05:00    1.514111
2012-03-11 05:30:00-04:00   -0.818680
2012-03-12 05:30:00-04:00   -0.097930
2012-03-13 05:30:00-04:00   -0.072169
2012-03-14 05:30:00-04:00    0.141429
Freq: D, dtype: float64

## 操作时区意识型（time zone-aware）Timestamp对象
跟时间序列和日期序列差不多，Timestamp对象也能被从单纯型（navie）本地化为time zone-aware，并从一个时区转换为另一个时区。

In [17]:
stamp = pd.Timestamp('2011-03-12 04:00')
stamp_utc = stamp.tz_localize('utc')
stamp_utc.tz_convert('US/Eastern')

Timestamp('2011-03-11 23:00:00-0500', tz='US/Eastern')

In [18]:
# 创建Timestamp时可以传入时区信息
stamp_moscow = pd.Timestamp('2011-03-12 04:00', tz='Europe/Moscow')
stamp_moscow

Timestamp('2011-03-12 04:00:00+0300', tz='Europe/Moscow')

In [25]:
# Timestamp的内在UTC时间戳（纳秒数）不会随时区的转换而变化
stamp_utc.value

1299902400000000000

In [20]:
stamp_utc.tz_convert('US/Eastern').value

1299902400000000000

In [21]:
# 30 minutes before DST transition
from pandas.tseries.offsets import Hour
stamp = pd.Timestamp('2012-03-12 01:30', tz='US/Eastern')
stamp

Timestamp('2012-03-12 01:30:00-0400', tz='US/Eastern')

In [22]:
stamp + Hour()

Timestamp('2012-03-12 02:30:00-0400', tz='US/Eastern')

In [23]:
# 90 minutes before DST transition
stamp = pd.Timestamp('2012-11-04 00:30', tz='US/Eastern')
stamp

Timestamp('2012-11-04 00:30:00-0400', tz='US/Eastern')

In [24]:
stamp + 2 * Hour()

Timestamp('2012-11-04 01:30:00-0500', tz='US/Eastern')

## 不同时区之间的运算

如果时间时间时区不同，那么结果就会是UTC时间，由于时间戳其实是以UTC储存的，索引计算很方便。

In [26]:
rng = pd.date_range('3/7/2012 9:30', periods=10, freq='B')
ts = Series(np.random.randn(len(rng)), index=rng)
ts

2012-03-07 09:30:00   -0.498195
2012-03-08 09:30:00    0.085745
2012-03-09 09:30:00    1.460264
2012-03-12 09:30:00    1.586622
2012-03-13 09:30:00   -0.083646
2012-03-14 09:30:00   -0.034900
2012-03-15 09:30:00   -0.262193
2012-03-16 09:30:00   -0.885324
2012-03-19 09:30:00    1.066322
2012-03-20 09:30:00    1.247435
Freq: B, dtype: float64

In [29]:
#注意naive是不能直接转换为时区的，必须先转换为localize再进行转换
ts1 = ts[:7].tz_localize('Europe/London')
ts2 = ts1[2:].tz_convert('Europe/Moscow')
result = ts1 + ts2

#自动转换为UTC
result.index

DatetimeIndex(['2012-03-07 09:30:00+00:00', '2012-03-08 09:30:00+00:00',
               '2012-03-09 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='B')